# Day 3 Solutions

_Note:_ I haven't included the results of running cells in this notebook because I no longer have access to the test cluster on which these exercises were run.

**Ex 3.1.3 Log into the IPython Notebook server on our cluster and create your own notebook.  Play around with the notebook for a few minutes to make sure everything works as you expect:**
1. **Import `numpy`, build a few arrays and perform some vector operations on them.**
2. **Import `matplotlib` and plot sin(x) from 0 to 2 $\pi$.**
3. **Import `pandas` and create a DataFrame with the GDP per capita of BE and NL.  Calculate the mean GDP per capita of BE and NL in the year 2003**
4. **Import `scikit-learn` and run the example of a linear regression on simulated data from yesterday morning**  _Note: this should also test out SciPy_

In [ ]:
2+2

In [ ]:
import numpy as np
A = np.array([1,2,3])
B = np.array([4,5,6])
C = A + 3*B
print(C)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

import math
x = np.linspace(0.0, 2*math.pi, 100)
y = np.sin(x)
plt.plot(x, y, 'r-')
plt.xlabel('Angle')
plt.ylabel('$\sin(x)$')
plt.title('Back to high-school')

In [ ]:
import pandas as pd
from pandas import DataFrame, Series

data = {
    'country': ['BE', 'BE', 'BE', 'NL', 'NL', 'NL'],
    'year': [1913, 1950, 2003, 1913, 1950, 2003],
    'gdp_per_capita': [4220, 5462, 21205, 4049, 5996, 21480]
}
frame = DataFrame(data)
frame

In [ ]:
frame[frame.year == 2003].gdp_per_capita.mean()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn import datasets, linear_model

In [ ]:
def linear_data_sample(N=40, rseed=0, m=3, b=-2):
    """Sample N points on the line y = m*x + b + err, where err is normally distributed"""
    rng = np.random.RandomState(rseed)

    x = 10 * rng.rand(N)
    y = m * x + b + 1.5*rng.randn(N)

    return x, y

In [ ]:
x, y = linear_data_sample()
plt.plot(x, y, 'o');

In [ ]:
# Least-squares linear Regression, the pedestrian way
def model(theta, x):
    b, m = theta
    return m * x + b

def square_deviation(theta, x, y):
    return np.sum((model(theta, x) - y) ** 2)

from scipy.optimize import fmin
theta_guess = [0, 1]
theta_fit = fmin(square_deviation, theta_guess, args=(x, y))

b_fit, m_fit = theta_fit
print 'Actual: m = 3, b = -2'
print 'Fit: m = %.2f, b = %.2f' % (m_fit, b_fit)

In [ ]:
regr = linear_model.LinearRegression(fit_intercept=True)
regr.fit(x.reshape((len(x), 1)), y)   # Scikit-learn expects x to be a column-vector

print 'Actual: m = 3, b = -2'
print 'Fit: m = %.2f, b = %.2f' % (regr.coef_[0], regr.intercept_)

In [ ]:
# predict y from the data
x_new = np.linspace(0, 10, 100)
y_new = regr.predict(x_new.reshape((len(x_new), 1)))  # Column vector

# plot the results
ax = plt.axes()
ax.scatter(x, y)
ax.plot(x_new, y_new)

ax.set_xlabel('x')
ax.set_ylabel('y')

ax.axis('tight');

**Ex 3.2.4: I've loaded all the names dataset into a single file `allnames.txt`, now with an extra year column.  The first few lines read:**
```
Mary,F,7065,1880
Anna,F,2604,1880
Emma,F,2003,1880
Elizabeth,F,1939,1880
```
**Try to read it into an RDD using `sc.textFile("file:///home/ipython/allnames.txt")`, then take the first 5 items.  What error message do you get?**

In [ ]:
rdd = sc.textFile('file:///home/ipython/allnames.txt')
rdd.take(5)

**Ex 3.1.5 Your first Spark job on the cluster!**
1. **Use Spark Core to load this file into an RDD and count the number of births between 1950 and 2000 with baby names starting in 'M'**
2. **Use Spark SQL and matplotlib to plot the number of boy and girl births for each year in the dataset**

In [ ]:
rdd = sc.textFile('names/allnames.txt')
rdd.cache()
rdd.take(5)

In [ ]:
(rdd
 .map(lambda line: line.split(','))
 .map(lambda fields: (fields[0], int(fields[2]), int(fields[3])))
 .filter(lambda (name, births, year): 1950 <= year <= 2000)
 .filter(lambda (name, births, year): name.startswith('M'))
 .map(lambda (name, births, year): births)
).sum()    # .sum() is the same as .reduce(lambda x, y: x + y)

In [ ]:
from pyspark.sql import SQLContext, Row
sqlCtx = SQLContext(sc)

In [ ]:
rowsRDD = (rdd
           .map(lambda line: line.split(','))
           .map(lambda fields: Row(name=fields[0], sex=fields[1], births=int(fields[2]), year=int(fields[3])))
           )
schemaRDD = sqlCtx.inferSchema(rowsRDD)   # In our cluster, we use Spark 1.5
schemaRDD.cache()                         # and inferSchema() will yield a deprecation warning
schemaRDD.registerTempTable("names")

In [ ]:
resultRDD = sqlCtx.sql("""
    SELECT
        year,
        SUM(CASE WHEN sex='M' THEN births ELSE 0 END) AS boys,
        SUM(CASE WHEN sex='F' THEN births ELSE 0 END) AS girls
    FROM names
    GROUP BY year
    ORDER BY year ASC
""")
result = resultRDD.map(lambda row: (row.year, row.boys, row.girls)).collect()
result[:10]

In [ ]:
years = [year for (year, _, _) in result]
girls = [girls for (_, girls, _) in result]
boys = [boys for (_, _, boys) in result]
plt.plot(years, girls, 'r-', label='Girls')
plt.plot(years, boys, 'b-', label='Boys')
plt.xlabel('Year')
plt.ylabel('Number of births')
plt.title('Babies born in US each year, by Gender')
plt.legend(loc='upper left')

**Ex 3.1.6 Saving your results to HDFS.  Say you want to get the complete list of unique names starting with 'M' from 1950 to 2000.  The list is quite large.**
1. **Write a Spark Core snippet to calculate that list in an RDD, but don't call `collect()` on it**
2. **Instead of using `collect()` to bring it all back to the driver node, you can use `saveAsTextFile('<yourname>.txt')` to write out the RDD to HDFS.  Do that, then use `sc.textFile()` and `take` to verify that the first few lines of the file look ok.**

In [ ]:
namesRDD = (rdd
 .map(lambda line: line.split(','))
 .map(lambda fields: (fields[0], int(fields[3])))
 .filter(lambda (name, year): 1950 <= year <= 2000)
 .filter(lambda (name, year): name.startswith('M'))
 .map(lambda (name, year): name)
)

In [ ]:
namesRDD.saveAsTextFile('patrick.txt')

In [ ]:
namesAgainRDD = sc.textFile('patrick.txt')
namesAgainRDD.take(5)

**Ex 3.1.8 When you load up `allnames.txt`, into how many partitions does Spark split the data?**

In [ ]:
# From the horse's mouth
sc.textFile('names/allnames.txt').getNumPartitions()

**Ex 3.1.9 You can use `repartition(N)` to force Spark to redistribute the data evenly across a different number of partitions.  With 30 MB, it makes little sense to use too many partitions.  But just for experimenting, repartition the data into 16 partitions, then run your code to count the number of births from 1950 to 2000 whose names start with 'M'.  Is is faster?  Slower?  The same?**

In [ ]:
from datetime import datetime
start = datetime.now()

(sc.textFile('names/allnames.txt')
 .map(lambda line: line.split(','))
 .map(lambda fields: (fields[0], int(fields[2]), int(fields[3])))
 .filter(lambda (name, births, year): 1950 <= year <= 2000)
 .filter(lambda (name, births, year): name.startswith('M'))
 .map(lambda (name, births, year): births)
).sum()    # .sum() is the same as .reduce(lambda x, y: x + y)

stop = datetime.now()
elapsed_s = (stop - start).total_seconds()
print("Without repartition, took {0:.2f} s".format(elapsed_s))


start = datetime.now()

(sc.textFile('names/allnames.txt')
 .repartition(16)
 .map(lambda line: line.split(','))
 .map(lambda fields: (fields[0], int(fields[2]), int(fields[3])))
 .filter(lambda (name, births, year): 1950 <= year <= 2000)
 .filter(lambda (name, births, year): name.startswith('M'))
 .map(lambda (name, births, year): births)
).sum()    # .sum() is the same as .reduce(lambda x, y: x + y)

stop = datetime.now()
elapsed_s = (stop - start).total_seconds()
print("With repartition, took {0:.2f} s".format(elapsed_s))

**Ex 3.1.11 If you can write SQL, you can write Hive.  Run a Hive query against your table to count the number of births between 1950 and 2000 whose names start with 'M'**

This will work:
```
SELECT
   SUM(births)
FROM hive_names_patrick
WHERE (year BETWEEN 1950 and 2000) AND (name LIKE 'M%');
```

**Ex 3.1.12 Use Spark Core to get all distinct names starting with 'M' from the year 1880 from the Hive table "hive_names_YOURNAME"**  
_WARNING_: Spark 1.3 onwards changed the type of Spark SQL objects from `SchemaRDD` to `DataFrame`, which has a different API.  To use the API that works in Spark 1.2, you have to get the `rdd` property of the table, e.g.:
```
tableRDD.rdd.filter(lambda row: row.year == 2000).count()
```

In [ ]:
from pyspark.sql import HiveContext, Row
hiveCtx = HiveContext(sc)

tablesRDD = hiveCtx.table("hive_names_patrick")

In [ ]:
tablesRDD.take(5)

In [ ]:
type(tablesRDD)

In [ ]:
(tablesRDD.rdd
 .filter(lambda row: row.year == 1880)
 .filter(lambda row: row.name.startswith('M'))
 .map(lambda row: row.name)
 .distinct()
).collect()

**Ex 3.1.13 Using `"CREATE TABLE hive_births_YOURNAME AS SELECT ..."`, create a table in Hive with the number of births per year.  Look up this table in Hue and verify that the data is accessible from outside your Spark session now**

Here's the raw command, just to see the data:

In [ ]:
hiveCtx.sql("SELECT year, SUM(births) FROM hive_names_patrick GROUP BY year ORDER BY year ASC").collect()

And here's the DDL statement:

In [ ]:
hiveCtx.sql("CREATE TABLE hive_births_patrick AS "
            "SELECT year, SUM(births) births FROM hive_names_patrick GROUP BY year ORDER BY year ASC")